In [ ]:
%pip install jsonlines tqdm

In [ ]:
import boto3
import botocore
import sagemaker 
import json
import jsonlines
import random
from tqdm import trange
from sagemaker.predictor import Predictor
from sagemaker import ModelPackage
import time
from time import gmtime, strftime
from datetime import datetime, timedelta
import uuid
import pandas as pd
import numpy as np
from sagemaker.model_monitor import (
    DefaultModelMonitor,
    DataCaptureConfig,
    CronExpressionGenerator,
    ModelQualityMonitor,
    EndpointInput,
)
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.model_monitor import DataCaptureConfig
from utils.monitoring_utils import run_model_monitor_job
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)
from urllib.parse import urlparse

sagemaker.__version__

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Fetched defaults config from location: /home/sagemaker-user/.config/sagemaker/config.yaml


'2.239.0'

In [3]:
sm = boto3.client("sagemaker")
s3 = boto3.client("s3")

In [4]:
session = sagemaker.Session()

In [5]:
pd.set_option("display.max_colwidth", None)

In [ ]:
%store -r 

%store

Stored variables and their in-db values:
baseline_s3_url                         -> 's3://sagemaker-us-east-1-906545278380/from-idea-t
bucket_name                             -> 'sagemaker-us-east-1-906545278380'
bucket_prefix                           -> 'from-idea-to-prod/xgboost'
dataset_feature_group_name              -> 'from-idea-to-prod-12-06-30-53'
dataset_file_local_path                 -> 'data/bank-additional/bank-additional-full.csv'
domain_id                               -> 'd-igloxuzrs3z2'
evaluation_s3_url                       -> 's3://sagemaker-us-east-1-906545278380/from-idea-t
experiment_name                         -> 'from-idea-to-prod-experiment-11-21-33-07'
feature_store_bucket_prefix             -> 'from-idea-to-prod/feature-store'
initialized                             -> True
input_s3_url                            -> 's3://sagemaker-us-east-1-906545278380/from-idea-t
mlflow_arn                              -> 'arn:aws:sagemaker:us-east-1:906545278380:mlflow

## Real-time inference data capture from a SageMaker endpoint

In [7]:
# List all deployed real-time endpoints. Depending on your existing environment you might have multiple endpoints
endpoints = sm.list_endpoints(StatusEquals="InService")["Endpoints"]
endpoint_name = ""

if not len(endpoints):
    print(f"There is no deployed active endpoints. You must have at least one endpoint. Run the step 3 pipeline to create a model")
else:
    endpoint_name = endpoints[0]['EndpointName']
    print(f"There are {len(endpoints)} active inference endpoints. Checking the data capture configuration:")
    
for ep in endpoints:
    print(f"Data capture configuration for {ep['EndpointName']}:")
    dcc = sm.describe_endpoint(EndpointName=ep['EndpointName']).get('DataCaptureConfig')
    print(f"{json.dumps(dcc, indent=2)}" if dcc else f"<No data capture config for this endpoint>")

There are 2 active inference endpoints. Checking the data capture configuration:
Data capture configuration for mlops-02-14-11-03-33-prod:
{
  "EnableCapture": true,
  "CaptureStatus": "Started",
  "CurrentSamplingPercentage": 80,
  "DestinationS3Uri": "s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod"
}
Data capture configuration for mlops-02-14-11-03-33-staging:
{
  "EnableCapture": true,
  "CaptureStatus": "Started",
  "CurrentSamplingPercentage": 100,
  "DestinationS3Uri": "s3://sagemaker-project-p-ca7phmcraepa/datacapture-staging"
}


### Deploy a model from the model registry as a real-time endpoint

In [ ]:
try:
    model_package_group = sm.describe_model_package_group(ModelPackageGroupName=model_package_group_name)
    print(f"There is model package group {model_package_group_name} in the model registry")
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException':
        print("******* ERROR *********")
        print(f"Model package group with the name {model_package_group_name} is not found. You need to run the step 3 pipeline to create a model")

In [ ]:
model_packages = []

# Find the latest model package
# Set the parameter ModelApprovalStatus='Approved' if you'd like to get only the approved packages
# Sort by the CreationTime
for p in sm.get_paginator('list_model_packages').paginate(
    ModelPackageGroupName=model_package_group_name,
    # ModelApprovalStatus='Approved',
    SortBy="CreationTime",
    SortOrder="Descending",
    ):
    model_packages.extend(p["ModelPackageSummaryList"])
    
if not len(model_packages):
    print("There is no model packages in the model package group {model_package_group_name}. You need to run the step 3 pipeline to create a model")
    
latest_model_package_arn = model_packages[0]['ModelPackageArn']
print(f"The most recent model package is version {model_packages[0]['ModelPackageVersion']}, {latest_model_package_arn}")


In [ ]:
r = sm.describe_model_package(ModelPackageName=latest_model_package_arn)

if r['ModelApprovalStatus'] not in ['Approved']:
    sm.update_model_package(
        ModelPackageArn=latest_model_package_arn,
        ModelApprovalStatus="Approved",
    )
else:
    print(f"Model approval status is {r['ModelApprovalStatus']} for the model package {latest_model_package_arn}")

In [ ]:
# Create a model from the registry using Python SDK
model = ModelPackage(role=sm_role, 
                     model_package_arn=model_packages[0]['ModelPackageArn'], 
                     sagemaker_session=session)

In [ ]:
endpoint_name = f"from-idea-to-prod-endpoint-{strftime('%d-%H-%M-%S', gmtime())}"

data_capture_config = DataCaptureConfig(
            enable_capture=True,
            sampling_percentage=100,
            destination_s3_uri=f"s3://{bucket_name}/{bucket_prefix}/data-capture",
            csv_content_types=["text/csv"],
        )

In [ ]:
# Deploy the model
model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    wait=False,
    data_capture_config=data_capture_config,
    endpoint_name=endpoint_name,
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.CSVDeserializer(),
)

In [ ]:
# Wait until the endpoint has the status InService, it takes approximately 5 min
waiter = session.sagemaker_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

### Check the data capture configuration

In [8]:
# Get the data capture configuration for the endpoint
# endpoint_name = "model-deploy-16-21-26-26-staging" # must be set before, but you can use any suitable endpoint

if not endpoint_name:
    print(f"You must have at least on endpoint with data capture configuration enabled!")
else:
    print(f"Checking the data capture configuration for the endpoint {endpoint_name}")
    data_capture_config = sm.describe_endpoint(EndpointName=endpoint_name)['DataCaptureConfig']
    data_capture_s3_url = data_capture_config['DestinationS3Uri']
    data_capture_bucket = data_capture_s3_url.split('/')[2]
    data_capture_prefix = '/'.join(data_capture_s3_url.split('/')[3:])

    print(json.dumps(data_capture_config, indent=2))
    print(f"Data capture S3 url: {data_capture_s3_url}")
    
    if not data_capture_config['EnableCapture']:
        print(f"Data capture config for the endpoint {endpoint_name} IS NOT ENABLED. You need to enable data capture for monitoring")

Checking the data capture configuration for the endpoint mlops-02-14-11-03-33-prod
{
  "EnableCapture": true,
  "CaptureStatus": "Started",
  "CurrentSamplingPercentage": 80,
  "DestinationS3Uri": "s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod"
}
Data capture S3 url: s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod


In [9]:
# Send data to the endpoint
def generate_endpoint_traffic(predictor, data):
    l = len(data)
    for i in trange(l):
        predictions = np.array(predictor.predict(data.iloc[i].values), dtype=float).squeeze()

In [10]:
# Get all file keys under a specified prefix
def get_file_list(bucket, prefix):
    try:
        files = [f.get("Key") for f in s3.list_objects(Bucket=bucket, Prefix=prefix).get("Contents")]
        print(f"Found {len(files)} files in s3://{bucket}/{prefix}")
        
        return files
    except TypeError:
        print(f"No files found in s3://{bucket}/{prefix}")
        return []

In [11]:
# Get S3 url for the latest captured data
def get_latest_data_capture_s3_url(bucket, prefix):
    capture_files = get_file_list(bucket, prefix)
    
    if capture_files:
        latest_data_capture_s3_url = f"s3://{bucket}/{'/'.join(capture_files[-1].split('/')[:-1])}"

        print(f"Latest data capture S3 url: {latest_data_capture_s3_url}")
        
        return latest_data_capture_s3_url
    else:
        return None

In [12]:
# Get S3 url for the latest monitoring job output
def get_latest_monitoring_report_s3_url(job_name):
    monitor_job = sm.list_processing_jobs(
        NameContains=job_name,
        SortOrder='Descending',
        MaxResults=2
    )['ProcessingJobSummaries'][0]['ProcessingJobName']

    monitoring_job_output_s3_url = sm.describe_processing_job(
        ProcessingJobName=monitor_job
    )['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri']

    print(f"Latest monitoring report S3 url: {monitoring_job_output_s3_url}")
    
    return monitoring_job_output_s3_url

In [13]:
# Helper to load a json file from S3
def load_json_from_file(file_s3_url):
    bucket = file_s3_url.split('/')[2]
    key = '/'.join(file_s3_url.split('/')[3:])
    print(f"Load JSON from: {bucket}/{key}")
    
    return json.loads(
        s3.get_object(Bucket=bucket, 
                      Key=key)["Body"].read().decode("utf-8")
    )

In [14]:
def get_latest_monitor_execution(monitor):
    mon_executions = monitor.list_executions()

    if len(mon_executions):
        latest_execution = mon_executions[-1]  # get the latest execution
        latest_execution.wait(logs=False)

        print(f"Latest execution status: {latest_execution.describe().get('ProcessingJobStatus')}")
        print(f"Latest execution result: {latest_execution.describe().get('ExitMessage')}")

        latest_job = latest_execution.describe()
        if latest_job["ProcessingJobStatus"] != "Completed":
            print("No completed executions to inspect further")
        else:
            report_uri = latest_execution.output.destination
            print(f"Report Uri: {report_uri}")
        
        return latest_execution
    else:
        print("No monitoring schedule executions found")
        return None

### Generate endpoint traffic and captured data

In [15]:
# Create a predictor class for the endpoint
predictor = Predictor(
    endpoint_name=endpoint_name, 
    serializer=sagemaker.serializers.CSVSerializer(),
    deserializer=sagemaker.deserializers.CSVDeserializer()
)

In [16]:
# If data capture is not enabled, enable it. 
# Note, that the call to update_data_capture_config updates the endpoint and it can take up to 10 min
if not data_capture_config['EnableCapture']:
    # Update data capture config for settings we use in this notebook
    data_capture_config = DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=data_capture_s3_url,
        csv_content_types=["text/csv"],
    )

    predictor.update_data_capture_config(data_capture_config)

Use test dataset prepared in the [step 2](02-sagemaker-containers.ipynb) or produced by the pipeline in the [step 3](02-sagemaker-pipeline.ipynb) notebooks and saved on the EFS volume:

In [17]:
!aws s3 cp $test_s3_url/test_x.csv tmp/test_x.csv
!aws s3 cp $test_s3_url/test_y.csv tmp/test_y.csv

download: s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/test/test_x.csv to tmp/test_x.csv
download: s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/test/test_y.csv to tmp/test_y.csv


In [18]:
# Set the number of data vectors from the test dataset sent to the inference endpoint
number_of_vectors = 100

In [19]:
test_x = pd.read_csv("tmp/test_x.csv", header=None).sample(number_of_vectors)

In [20]:
test_x.head()

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
1073,31,2,999,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
1294,57,1,999,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1723,34,3,999,1,1,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
2357,41,2,999,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1563,35,2,999,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [21]:
test_x.columns = [f'_c{i}' for i in range(len(test_x.columns))]

In [22]:
test_x.head(1)

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c49,_c50,_c51,_c52,_c53,_c54,_c55,_c56,_c57,_c58
1073,31,2,999,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0


In [24]:
# test_x.dtypes

Send the data to the endpoint:

In [25]:
generate_endpoint_traffic(predictor, test_x)

100%|██████████| 100/100 [00:01<00:00, 56.81it/s]


In [28]:
# If you run this notebook not the first time, there might be some data capture files from the previous runs
# We recommend to delete all existing files under the data capture S3 path to avoid any inconsistences
# Uncomment and run the following line to delete all files under the data capture S3 path

# !aws s3 rm {data_capture_s3_url} --recursive

### View captured data

In [31]:
# You might need to execute this line several times until you see the captured data
!aws s3 ls {data_capture_s3_url} --recursive

2025-02-15 21:30:29      34074 datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/15/21/29-20-227-06f6a337-a6ae-419f-9af0-be9a6cdc54a6.jsonl


In [32]:
capture_files = get_file_list(data_capture_bucket, data_capture_prefix)

Found 1 files in s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod


In [33]:
assert len(capture_files) > 0, "Wait until the capture data delivered to the Amazon S3 bucket"

In [34]:
capture_files[0]

'datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/15/21/29-20-227-06f6a337-a6ae-419f-9af0-be9a6cdc54a6.jsonl'

In [35]:
# Download a capture data file and print it's content
file_key = capture_files[-1]
S3Downloader.download(f"s3://{data_capture_bucket}/{file_key}", f"./tmp")

print(f"Content of the capture file:")
# Read the jsonl file and show the first object
with jsonlines.open(f"./tmp/{file_key.split('/')[-1]}") as reader:      
    print(json.dumps(reader.read(), indent=2))
    # print(json.dumps(reader.read(), indent=2))

Content of the capture file:
{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "31,2,999,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0.1255551427602768\n",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "b1bfa55b-1ea7-451f-b531-6f844e5af789",
    "inferenceTime": "2025-02-15T21:29:20Z"
  },
  "eventVersion": "0"
}


## Monitor data quality

### Create a baselining job with the training dataset

In [36]:
!aws s3 ls {baseline_s3_url}/

                           PRE reports/
                           PRE results/
2025-02-15 11:38:16    4982068 baseline.csv


In [37]:
baseline_results_s3_url = f"{baseline_s3_url}/results"
data_mon_reports_s3_url = f"{baseline_s3_url}/reports"
baseline_dataset_uri = f"{baseline_s3_url}/baseline.csv"

In [38]:
data_monitor = DefaultModelMonitor(
    role=sm_role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
    sagemaker_session=session,
)

[02/15/25 21:44:41] INFO     Ignoring unnecessary instance type: None.                            ]8;id=346880;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=154690;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#528\528]8;;\

In [39]:
data_baseline_job_name = f"from-idea-to-prod-data-baselining-{strftime('%d-%H-%M-%S', gmtime())}-{str(uuid.uuid4())[:8]}"

data_baseline_job = data_monitor.suggest_baseline(
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=baseline_results_s3_url,
    wait=False,
    logs=False,
    job_name=data_baseline_job_name,
)

print(data_baseline_job_name)

[02/15/25 21:44:44] INFO     Creating processing-job with name                                      ]8;id=712023;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=22391;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#1575\1575]8;;\
                             from-idea-to-prod-data-baselining-15-21-44-42-7cfad4f6                                

from-idea-to-prod-data-baselining-15-21-44-42-7cfad4f6


In [40]:
data_baseline_job.wait(logs=False)

...........................................................!

### See the generated statistics and constraints

In [41]:
data_monitor.describe_latest_baselining_job()

{'ProcessingInputs': [{'InputName': 'baseline_dataset_input',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/baseline.csv',
    'LocalPath': '/opt/ml/processing/input/baseline_dataset_input',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'monitoring_output',
    'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/results',
     'LocalPath': '/opt/ml/processing/output',
     'S3UploadMode': 'EndOfJob'},
    'AppManaged': False}]},
 'ProcessingJobName': 'from-idea-to-prod-data-baselining-15-21-44-42-7cfad4f6',
 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 1,
   'InstanceType': 'ml.m5.xlarge',
   'VolumeSizeInGB': 20}},
 'StoppingCondition': {'MaxRuntimeInSeconds': 3600},
 'AppSpecification': {'ImageUri': '156

In [42]:
!aws s3 ls {baseline_results_s3_url}/

2025-02-15 21:48:21       9368 constraints.json
2025-02-15 21:48:21    1496024 statistics.json


In [43]:
data_statistics_s3_url = f"{baseline_results_s3_url}/statistics.json"
data_constraints_s3_url = f"{baseline_results_s3_url}/constraints.json"

In [44]:
!aws s3 cp {data_constraints_s3_url} ./tmp/
!aws s3 cp {data_statistics_s3_url} ./tmp/

download: s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/results/constraints.json to tmp/constraints.json
download: s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/results/statistics.json to tmp/statistics.json


In [45]:
!head -20 tmp/constraints.json

{
  "version" : 0.0,
  "features" : [ {
    "name" : "_c0",
    "inferred_type" : "Integral",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
    }
  }, {
    "name" : "_c1",
    "inferred_type" : "Integral",
    "completeness" : 1.0,
    "num_constraints" : {
      "is_non_negative" : true
    }
  }, {
    "name" : "_c2",
    "inferred_type" : "Integral",
    "completeness" : 1.0,


In [46]:
!head -20 tmp/statistics.json

{
  "version" : 0.0,
  "dataset" : {
    "item_count" : 41188
  },
  "features" : [ {
    "name" : "_c0",
    "inferred_type" : "Integral",
    "numerical_statistics" : {
      "common" : {
        "num_present" : 41188,
        "num_missing" : 0
      },
      "mean" : 40.02406040594348,
      "sum" : 1648511.0,
      "std_dev" : 10.421123471838728,
      "min" : 17.0,
      "max" : 98.0,
      "approximate_num_distinct_values" : 82,
      "completeness" : 1.0,


In [ ]:
baseline_job = data_monitor.latest_baselining_job
statistics_df = pd.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
statistics_df.head()

In [ ]:
constraints_df = pd.json_normalize(
    baseline_job.suggested_constraints().body_dict["features"]
)
constraints_df

For this dataset the baselining job suggest three constraints:
1. DataType
2. Completeness
3. Is non-negative

### Create a data monitoring schedule

In [49]:
# uncomment to see the file
# !pygmentize ./record_preprocessor.py

In [50]:
# Upload the preprocessing script to S3
record_preprocessor_s3_url = f"s3://{bucket_name}/{bucket_prefix}/code"

In [51]:
!aws s3 cp ./record_preprocessor.py {record_preprocessor_s3_url}/

upload: ./record_preprocessor.py to s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/code/record_preprocessor.py


In [52]:
data_mon_schedule_name = "from-idea-to-prod-data-monitor-schedule-" + strftime(
    "%Y-%m-%d-%H-%M-%S", gmtime()
)

data_monitor.create_monitoring_schedule(
    monitor_schedule_name=data_mon_schedule_name,
    endpoint_input=predictor.endpoint_name,
    record_preprocessor_script=f"{record_preprocessor_s3_url}/record_preprocessor.py",
    # post_analytics_processor_script=s3_code_postprocessor_uri,
    output_s3_uri=data_mon_reports_s3_url,
    statistics=data_monitor.baseline_statistics(),
    constraints=data_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

[02/15/25 21:52:23] INFO     Creating Monitoring Schedule with name:                       ]8;id=716322;file:///opt/conda/lib/python3.11/site-packages/sagemaker/model_monitor/model_monitoring.py\model_monitoring.py]8;;\:]8;id=631884;file:///opt/conda/lib/python3.11/site-packages/sagemaker/model_monitor/model_monitoring.py#1560\1560]8;;\
                             from-idea-to-prod-data-monitor-schedule-2025-02-15-21-52-22                           

In [53]:
while data_monitor.describe_schedule()["MonitoringScheduleStatus"] != "Scheduled":
    print(f"Waiting until data monitoring schedule status becomes Scheduled")
    time.sleep(3)

data_monitor.describe_schedule()

Waiting until data monitoring schedule status becomes Scheduled
Waiting until data monitoring schedule status becomes Scheduled
Waiting until data monitoring schedule status becomes Scheduled


{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:906545278380:monitoring-schedule/from-idea-to-prod-data-monitor-schedule-2025-02-15-21-52-22',
 'MonitoringScheduleName': 'from-idea-to-prod-data-monitor-schedule-2025-02-15-21-52-22',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'DataQuality',
 'CreationTime': datetime.datetime(2025, 2, 15, 21, 52, 23, 796000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 2, 15, 21, 52, 31, 672000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'data-quality-job-definition-2025-02-15-21-52-23-138',
  'MonitoringType': 'DataQuality'},
 'EndpointName': 'mlops-02-14-11-03-33-prod',
 'ResponseMetadata': {'RequestId': '8821ec86-7f9c-49cf-8228-a3f62689d9fd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8821ec86-7f9c-49cf-8228-a3f62689d9fd',
   'content-type': 'application/x-amz-json-1.1',
   'content-leng

### Generate compliant traffic

In [54]:
test_x

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c49,_c50,_c51,_c52,_c53,_c54,_c55,_c56,_c57,_c58
1073,31,2,999,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
1294,57,1,999,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1723,34,3,999,1,1,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
2357,41,2,999,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1563,35,2,999,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3885,27,1,999,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0
3990,46,4,999,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3306,62,1,16,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2366,33,1,999,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0


In [55]:
generate_endpoint_traffic(predictor, test_x)

100%|██████████| 100/100 [00:01<00:00, 74.33it/s]


### See the captured data

In [56]:
# If you run this notebook not the first time, there might be some data capture files from the previous runs
# We recommend to delete all previous captured files under the data capture S3 path to avoid any inconsistences
# Uncomment and run the following line to delete all files under the data capture S3 path

# !aws s3 rm {data_capture_s3_url} --recursive

In [59]:
!aws s3 ls {data_capture_s3_url} --recursive

2025-02-15 21:30:29      34074 datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/15/21/29-20-227-06f6a337-a6ae-419f-9af0-be9a6cdc54a6.jsonl
2025-02-15 21:53:39      39429 datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/15/21/52-33-432-72237081-a8ae-45ba-ae6d-320adab0a782.jsonl


### Launch a manual monitoring job

In [60]:
# uncomment to see the code
# !pygmentize ./utils/monitoring_utils.py

In [61]:
latest_data_capture_s3_url = get_latest_data_capture_s3_url(data_capture_bucket, data_capture_prefix)

Found 2 files in s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod
Latest data capture S3 url: s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/15/21


In [62]:
print(f"Data capture path: {latest_data_capture_s3_url}")
print(f"Data baseline statistics file: {data_statistics_s3_url}")
print(f"Data baseline constraints file: {data_constraints_s3_url}")
print(f"Data monitor report output path: {data_mon_reports_s3_url}")
print(f"Record preprocessor script path: {record_preprocessor_s3_url}")

Data capture path: s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/15/21
Data baseline statistics file: s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/results/statistics.json
Data baseline constraints file: s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/results/constraints.json
Data monitor report output path: s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/reports
Record preprocessor script path: s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/code


In [63]:
from utils.monitoring_utils import run_model_monitor_job

run_model_monitor_job(
    region=region,
    instance_type="ml.m5.xlarge",
    role=sm_role,
    data_capture_path=latest_data_capture_s3_url,
    statistics_path=data_statistics_s3_url,
    constraints_path=data_constraints_s3_url,
    reports_path=data_mon_reports_s3_url,
    instance_count=1,
    preprocessor_path=f"{record_preprocessor_s3_url}/record_preprocessor.py",
    postprocessor_path=None,
    publish_cloudwatch_metrics="Disabled",
    logs=False,
)

[02/15/25 21:58:27] INFO     Creating processing-job with name                                      ]8;id=731689;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=388987;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#1575\1575]8;;\
                             sagemaker-model-monitor-analyzer-2025-02-15-21-58-26-656                              

...........................................................!

### See the monitoring job output

In [64]:
manual_monitoring_job_output_s3_url = get_latest_monitoring_report_s3_url("sagemaker-model-monitor-analyzer")

Latest monitoring report S3 url: s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/reports/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/15/21


In [65]:
!aws s3 ls {manual_monitoring_job_output_s3_url}/

2025-02-15 22:01:55        747 constraint_violations.json
2025-02-15 22:01:53       9374 constraints.json
2025-02-15 22:01:53     145625 statistics.json


In [66]:
violations = load_json_from_file(f"{manual_monitoring_job_output_s3_url}/constraint_violations.json")

Load JSON from: sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/reports/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/15/21/constraint_violations.json


In [67]:
pd.json_normalize(violations["violations"])

,feature_name,constraint_check_type,description
0,_c0,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."
1,_c12,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."
2,_c1,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."


In [68]:
!aws s3 cp {manual_monitoring_job_output_s3_url}/constraint_violations.json ./tmp/

download: s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/reports/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/15/21/constraint_violations.json to tmp/constraint_violations.json


In [69]:
!head ./tmp/constraint_violations.json

{
  "violations" : [ {
    "feature_name" : "_c0",
    "constraint_check_type" : "data_type_check",
    "description" : "Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."
  }, {
    "feature_name" : "_c12",
    "constraint_check_type" : "data_type_check",
    "description" : "Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."
  }, {


In [70]:
statistics = load_json_from_file(f"{manual_monitoring_job_output_s3_url}/statistics.json")
constraints = load_json_from_file(f"{manual_monitoring_job_output_s3_url}/constraints.json")

print(f"Records processed: {statistics['dataset']['item_count']}")

Load JSON from: sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/reports/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/15/21/statistics.json
Load JSON from: sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/reports/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/15/21/constraints.json
Records processed: 151


In [ ]:
pd.json_normalize(statistics["features"]).head()

In [72]:
pd.json_normalize(constraints["features"]).head()

,name,inferred_type,completeness,num_constraints.is_non_negative
0,_c0,Fractional,1.0,True
1,_c1,Fractional,1.0,True
2,_c2,Integral,1.0,True
3,_c3,Integral,1.0,True
4,_c4,Integral,1.0,True


### What is monitored?

In [73]:
with open("tmp/constraints.json", "r") as c:
    data = c.read()
    
print(json.dumps(json.loads(data)["monitoring_config"], indent=2))

{
  "evaluate_constraints": "Enabled",
  "emit_metrics": "Enabled",
  "datatype_check_threshold": 1.0,
  "domain_content_threshold": 1.0,
  "distribution_constraints": {
    "perform_comparison": "Enabled",
    "comparison_threshold": 0.1,
    "comparison_method": "Robust",
    "categorical_comparison_threshold": 0.1,
    "categorical_drift_method": "LInfinity"
  }
}


### Generate non-compliant traffic

In [74]:
non_compliant_pd = test_x.copy()
non_compliant_pd.iloc[:,0] = -1.0

In [75]:
non_compliant_pd.head()

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,...,_c49,_c50,_c51,_c52,_c53,_c54,_c55,_c56,_c57,_c58
1073,-1,2,999,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
1294,-1,1,999,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1723,-1,3,999,1,1,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
2357,-1,2,999,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1563,-1,2,999,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [76]:
# Remove previous saved data capture from the S3 bucket
latest_data_capture_s3_url = get_latest_data_capture_s3_url(data_capture_bucket, data_capture_prefix)

Found 2 files in s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod
Latest data capture S3 url: s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/15/21


In [77]:
# If you run this notebook not the first time, there might be some data capture files from the previous runs
# We recommend to delete all existing files under the data capture S3 path to avoid any inconsistences
# Uncomment and run the following line to delete all files under the data capture S3 path

# !aws s3 rm {latest_data_capture_s3_url} --recursive

In [78]:
generate_endpoint_traffic(predictor, non_compliant_pd)

100%|██████████| 100/100 [00:01<00:00, 95.06it/s]


### See the captured data

In [80]:
!aws s3 ls {data_capture_s3_url} --recursive

2025-02-15 21:30:29      34074 datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/15/21/29-20-227-06f6a337-a6ae-419f-9af0-be9a6cdc54a6.jsonl
2025-02-15 21:53:39      39429 datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/15/21/52-33-432-72237081-a8ae-45ba-ae6d-320adab0a782.jsonl
2025-02-16 11:20:09      34051 datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/11/19-01-920-70f9078e-1765-4c8f-9f9c-1845bd3e6de4.jsonl


In [81]:
capture_files = get_file_list(data_capture_bucket, data_capture_prefix)

Found 3 files in s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod


In [82]:
# Download a capture data file and print it's content
file_key = capture_files[-1]
S3Downloader.download(f"s3://{data_capture_bucket}/{file_key}", f"./tmp")

print(f"Content of the capture file:")
# Read the jsonl file and show the first object
with jsonlines.open(f"./tmp/{file_key.split('/')[-1]}") as reader:      
    print(json.dumps(reader.read(), indent=2))

Content of the capture file:
{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "-1,2,999,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0.2893185019493103\n",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "a51faf5d-da3e-400d-b776-f02ac8725d6c",
    "inferenceTime": "2025-02-16T11:19:01Z"
  },
  "eventVersion": "0"
}


### Launch a manual monitoring job

In [83]:
latest_data_capture_s3_url = get_latest_data_capture_s3_url(data_capture_bucket, data_capture_prefix)

Found 3 files in s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod
Latest data capture S3 url: s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/11


In [84]:
run_model_monitor_job(
    region=region,
    instance_type="ml.m5.xlarge",
    role=sm_role,
    data_capture_path=latest_data_capture_s3_url,
    statistics_path=data_statistics_s3_url,
    constraints_path=data_constraints_s3_url,
    reports_path=data_mon_reports_s3_url,
    instance_count=1,
    preprocessor_path=f"{record_preprocessor_s3_url}/record_preprocessor.py",
    postprocessor_path=None,
    publish_cloudwatch_metrics="Disabled",
    logs=False,
)

[02/16/25 14:34:01] INFO     Creating processing-job with name                                      ]8;id=168004;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=612574;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#1575\1575]8;;\
                             sagemaker-model-monitor-analyzer-2025-02-16-14-34-01-134                              

...........................................................!

### See the monitoring job output

In [85]:
manual_monitoring_job_output_s3_url = get_latest_monitoring_report_s3_url("sagemaker-model-monitor-analyzer")

Latest monitoring report S3 url: s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/reports/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/11


In [86]:
!aws s3 ls {manual_monitoring_job_output_s3_url}/

2025-02-16 14:37:26        747 constraint_violations.json
2025-02-16 14:37:24       9375 constraints.json
2025-02-16 14:37:24     121226 statistics.json


In [87]:
violations = load_json_from_file(f"{manual_monitoring_job_output_s3_url}/constraint_violations.json")
violations

Load JSON from: sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/reports/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/11/constraint_violations.json


{'violations': [{'feature_name': '_c0',
   'constraint_check_type': 'data_type_check',
   'description': 'Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral.'},
  {'feature_name': '_c12',
   'constraint_check_type': 'data_type_check',
   'description': 'Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral.'},
  {'feature_name': '_c1',
   'constraint_check_type': 'data_type_check',
   'description': 'Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral.'}]}

In [88]:
pd.json_normalize(violations["violations"])

,feature_name,constraint_check_type,description
0,_c0,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."
1,_c12,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."
2,_c1,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."


In [89]:
statistics = load_json_from_file(f"{manual_monitoring_job_output_s3_url}/statistics.json")
constraints = load_json_from_file(f"{manual_monitoring_job_output_s3_url}/constraints.json")

print(f"Records processed: {statistics['dataset']['item_count']}")

Load JSON from: sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/reports/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/11/statistics.json
Load JSON from: sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/reports/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/11/constraints.json
Records processed: 70


In [ ]:
pd.json_normalize(statistics["features"]).head()

In [92]:
pd.json_normalize(constraints["features"]).head()

,name,inferred_type,completeness,num_constraints.is_non_negative
0,_c0,Fractional,1.0,False
1,_c1,Fractional,1.0,True
2,_c2,Integral,1.0,True
3,_c3,Integral,1.0,True
4,_c4,Integral,1.0,True


### List schedule executions and monitoring reports

List executions and view a monitoring job details:

In [93]:
latest_execution = get_latest_monitor_execution(data_monitor)

!Latest execution status: Completed
Latest execution result: CompletedWithViolations: Job completed successfully with 3 violations.
Report Uri: s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/baseline/reports/mlops-02-14-11-03-33-prod/from-idea-to-prod-data-monitor-schedule-2025-02-15-21-52-22/2025/02/16/12


See details about the latest scheduled monitoring execution:

In [ ]:
# this line fails if there are no executions
latest_execution.describe()

Get the latest execution statistics and constraint violations as objects:

In [95]:
last_execution_statistics = latest_execution.statistics()
last_execution_violations = latest_execution.constraint_violations()

Load reports into Pandas DataFrame:

In [ ]:
pd.json_normalize(last_execution_statistics.body_dict["features"]).head()

In [98]:
pd.json_normalize(last_execution_violations.body_dict["violations"]).head()

,feature_name,constraint_check_type,description
0,_c0,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."
1,_c12,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."
2,_c1,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."


See the baseline and the latest data profiling statistics:

In [ ]:
pd.json_normalize(data_monitor.baseline_statistics().body_dict["features"]).head()

In [ ]:
pd.json_normalize(data_monitor.latest_monitoring_statistics().body_dict["features"]).head()

#### View a violation report

In [102]:
violations = data_monitor.latest_monitoring_constraint_violations()

In [103]:
violations

In [104]:
if not violations:
    print("No constraint violations report found")
else:
    violations_df = pd.json_normalize(violations.body_dict["violations"]).head()

In [105]:
violations_df

,feature_name,constraint_check_type,description
0,_c0,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."
1,_c12,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."
2,_c1,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."


---

## Monitor model quality

### Define helper functions

In [113]:
def generate_ground_truth_with_id(inference_id):
    # set random seed to get consistent results
    random.seed(inference_id) 
    rand = random.random()
    
    # format required by the merge container.
    return {
        "groundTruthData": {
            "data": "0" if rand < 0.5 else "1", #str(rand),
            "encoding": "CSV",
        },
        "eventMetadata": {
            "eventId": str(inference_id), # eventId must correlate with the eventId in the data capture file
        },
        "eventVersion": "0",
    }

In [114]:
def upload_ground_truth(ground_truth_upload_s3_url, file_name, records, upload_time):
    target_s3_uri = f"{ground_truth_upload_s3_url}/{upload_time:%Y/%m/%d/%H}/{file_name}"
    number_of_records = len(records.split('\n'))
    print(f"Uploading {number_of_records} records to {target_s3_uri}")
    
    S3Uploader.upload_string_as_file_body(records, target_s3_uri)
    
    return target_s3_uri

### Create a model quality monitor

In [115]:
model_monitor = ModelQualityMonitor(
    role=sm_role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=session
)

[02/16/25 14:53:57] INFO     Ignoring unnecessary instance type: None.                            ]8;id=305083;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=624150;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#528\528]8;;\

### Run a model quality baseline job

In [116]:
!aws s3 ls {prediction_baseline_s3_url}/

                           PRE reports/
                           PRE results/
2025-02-12 06:58:25      63027 prediction_baseline.csv


In [117]:
prediction_baseline_results_s3_url = f"{prediction_baseline_s3_url}/results"
model_mon_reports_s3_url = f"{prediction_baseline_s3_url}/reports"
prediction_baseline_dataset_uri = f"{prediction_baseline_s3_url}/prediction_baseline.csv"

In [118]:
model_baseline_job_name = f"from-idea-to-prod-model-baselining-{strftime('%d-%H-%M-%S', gmtime())}-{str(uuid.uuid4())[:8]}"

model_baseline_job = model_monitor.suggest_baseline(
    baseline_dataset=prediction_baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri = prediction_baseline_results_s3_url, 
    problem_type="BinaryClassification",
    inference_attribute= "prediction", # The column in the dataset that contains predictions
    probability_attribute= "probability", # The column in the dataset that contains probabilities
    ground_truth_attribute= "label", # The column in the dataset that contains ground truth labels
    job_name=model_baseline_job_name,
)

print(model_baseline_job_name)

[02/16/25 14:54:01] INFO     Creating processing-job with name                                      ]8;id=889937;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=190844;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#1575\1575]8;;\
                             from-idea-to-prod-model-baselining-16-14-54-00-686ddf96                               

from-idea-to-prod-model-baselining-16-14-54-00-686ddf96


In [119]:
model_baseline_job.wait(logs=False)

...........................................................!

### Inspect the generated baseline statistics and constraints


In [121]:
!aws s3 ls {prediction_baseline_results_s3_url}/

2025-02-16 14:57:22       1372 constraints.json
2025-02-16 14:57:22      78532 statistics.json


In [122]:
latest_model_baseline_job = model_monitor.latest_baselining_job
pd.DataFrame(latest_model_baseline_job.suggested_constraints().body_dict["binary_classification_constraints"]).T

,threshold,comparison_operator
recall,0.222917,LessThanThreshold
precision,0.644578,LessThanThreshold
accuracy,0.89512,LessThanThreshold
true_positive_rate,0.222917,LessThanThreshold
true_negative_rate,0.983787,LessThanThreshold
false_positive_rate,0.016213,GreaterThanThreshold
false_negative_rate,0.777083,GreaterThanThreshold
auc,0.772685,LessThanThreshold
f0_5,0.467657,LessThanThreshold
f1,0.331269,LessThanThreshold


In [123]:
pd.DataFrame(latest_model_baseline_job.baseline_statistics().body_dict["binary_classification_metrics"]["confusion_matrix"])

,0,1
0,3580,373
1,59,107


In [ ]:
pd.json_normalize(latest_model_baseline_job.baseline_statistics().body_dict["binary_classification_metrics"]).T

### Generate endpoint traffic
Generate synthetic traffic to the endpoint to capture inference input and output.

In [125]:
# Remove previous data capture saved to the S3 bucket
latest_data_capture_s3_url = get_latest_data_capture_s3_url(data_capture_bucket, data_capture_prefix)

Found 3 files in s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod
Latest data capture S3 url: s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/11


In [126]:
# If you run this notebook not the first time, there might be some data capture files from the previous runs
# We recommend to delete all existing files under the data capture S3 path to avoid any inconsistences
# Uncomment and run the following line to delete all files under the data capture S3 path

# !aws s3 rm {latest_data_capture_s3_url} --recursive

In [127]:
test_x.shape

(100, 59)

In [128]:
generate_endpoint_traffic(predictor, test_x)

100%|██████████| 100/100 [00:00<00:00, 100.50it/s]


In [131]:
!aws s3 ls {data_capture_s3_url} --recursive

2025-02-15 21:30:29      34074 datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/15/21/29-20-227-06f6a337-a6ae-419f-9af0-be9a6cdc54a6.jsonl
2025-02-15 21:53:39      39429 datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/15/21/52-33-432-72237081-a8ae-45ba-ae6d-320adab0a782.jsonl
2025-02-16 11:20:09      34051 datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/11/19-01-920-70f9078e-1765-4c8f-9f9c-1845bd3e6de4.jsonl
2025-02-16 15:01:49      37480 datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/15/00-44-130-fcebff3a-f559-47ea-beeb-98c6a0155f8a.jsonl


### Ingest ground truth data

In [132]:
# Set the S3 url where to store the ground truth labels
variant_name = sm.describe_endpoint(EndpointName=predictor.endpoint_name)["ProductionVariants"][0]["VariantName"]
ground_truth_upload_s3_url = f"s3://{data_capture_bucket}/ground_truth_data/{predictor.endpoint_name}/{variant_name}"
ground_truth_upload_s3_url

's3://sagemaker-project-p-ca7phmcraepa/ground_truth_data/mlops-02-14-11-03-33-prod/AllTraffic'

In [133]:
# Get the S3 prefix where the latest capture data has been delivered
latest_data_capture_s3_url = get_latest_data_capture_s3_url(data_capture_bucket, data_capture_prefix)
latest_data_capture_prefix = '/'.join(latest_data_capture_s3_url.split('/')[3:])

Found 4 files in s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod
Latest data capture S3 url: s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/15


In [134]:
# Get the list of capture data file prefixes in the latest capture data location
capture_files = get_file_list(data_capture_bucket, latest_data_capture_prefix)

assert capture_files, f"No capture data files found in {latest_data_capture_prefix}. Generate endpoint traffic and wait until capture data appears in the bucket!"

# For each capture data file get the eventIds and generate correlated ground truth labels
for f in capture_files:
    f_name = f.split('/')[-1]
    
    print(f"Downloading {f}")
    S3Downloader.download(f"s3://{data_capture_bucket}/{f}", "./tmp")
    
    print(f"Reading inference ids from the file: ./tmp/{f_name}")
    with jsonlines.open(f"./tmp/{f_name}") as reader: 
        ground_truth_records = "\n".join([
            json.dumps(r) for r in [generate_ground_truth_with_id(l["eventMetadata"]["eventId"]) for l in reader]
        ])
    lastest_ground_truth_s3_uri = upload_ground_truth(ground_truth_upload_s3_url, f"gt-{f_name}", ground_truth_records, datetime.utcnow())

Found 1 files in s3://sagemaker-project-p-ca7phmcraepa/datacapture-prod/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/15
Reading inference ids from the file: ./tmp/00-44-130-fcebff3a-f559-47ea-beeb-98c6a0155f8a.jsonl
Uploading 77 records to s3://sagemaker-project-p-ca7phmcraepa/ground_truth_data/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/15/gt-00-44-130-fcebff3a-f559-47ea-beeb-98c6a0155f8a.jsonl


In [135]:
# List uploaded ground truth files
!aws s3 ls {ground_truth_upload_s3_url} --recursive

2025-02-16 15:02:13      11164 ground_truth_data/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/15/gt-00-44-130-fcebff3a-f559-47ea-beeb-98c6a0155f8a.jsonl


Download the last ingested ground truth data file and see it's content:

In [136]:
# Download the last ground truth file to Studio's EFS
!aws s3 cp {lastest_ground_truth_s3_uri} ./tmp/groundtruth.jsonl

download: s3://sagemaker-project-p-ca7phmcraepa/ground_truth_data/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/15/gt-00-44-130-fcebff3a-f559-47ea-beeb-98c6a0155f8a.jsonl to tmp/groundtruth.jsonl


In [137]:
!head ./tmp/groundtruth.jsonl

{"groundTruthData": {"data": "0", "encoding": "CSV"}, "eventMetadata": {"eventId": "88fdfecc-1a1a-4de9-b58e-70ce95a69c39"}, "eventVersion": "0"}
{"groundTruthData": {"data": "0", "encoding": "CSV"}, "eventMetadata": {"eventId": "73a04cf8-1912-4559-8e5f-785dd7a58c4a"}, "eventVersion": "0"}
{"groundTruthData": {"data": "0", "encoding": "CSV"}, "eventMetadata": {"eventId": "95b329ea-1729-4fbb-b455-9df943190b4f"}, "eventVersion": "0"}
{"groundTruthData": {"data": "0", "encoding": "CSV"}, "eventMetadata": {"eventId": "dbb1d621-4404-48a2-ac79-7057594cdbc5"}, "eventVersion": "0"}
{"groundTruthData": {"data": "0", "encoding": "CSV"}, "eventMetadata": {"eventId": "6dde1030-eb16-4621-b540-e4d85fb6e4f5"}, "eventVersion": "0"}
{"groundTruthData": {"data": "1", "encoding": "CSV"}, "eventMetadata": {"eventId": "649df323-dc0d-480b-8c72-8be8faa81ba7"}, "eventVersion": "0"}
{"groundTruthData": {"data": "0", "encoding": "CSV"}, "eventMetadata": {"eventId": "76f842be-14a8-4a3b-a850-d7eaa722b9ba"}, "event

### Create a model monitoring schedule

In [138]:
endpoint_input = EndpointInput(
    endpoint_name=predictor.endpoint_name,
    probability_attribute="0",
    probability_threshold_attribute=0.5,
    destination="/opt/ml/processing/input_data",
)

In [139]:
model_mon_schedule_name = "from-idea-to-prod-model-monitor-schedule-" + strftime(
    "%Y-%m-%d-%H-%M-%S", gmtime()
)

model_monitor.create_monitoring_schedule(
    monitor_schedule_name=model_mon_schedule_name,
    endpoint_input=endpoint_input,
    problem_type="BinaryClassification",
    # record_preprocessor_script=f"{record_preprocessor_s3_url}/record_preprocessor.py",
    # post_analytics_processor_script=s3_code_postprocessor_uri,
    output_s3_uri=model_mon_reports_s3_url,
    ground_truth_input=ground_truth_upload_s3_url,
    constraints=model_monitor.suggested_constraints() if model_monitor.latest_baselining_job else f"{prediction_baseline_results_s3_url}/constraints.json",
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

[02/16/25 15:02:28] INFO     Creating Monitoring Schedule with name:                       ]8;id=185324;file:///opt/conda/lib/python3.11/site-packages/sagemaker/model_monitor/model_monitoring.py\model_monitoring.py]8;;\:]8;id=139315;file:///opt/conda/lib/python3.11/site-packages/sagemaker/model_monitor/model_monitoring.py#1560\1560]8;;\
                             from-idea-to-prod-model-monitor-schedule-2025-02-16-15-02-27                          

In [140]:
while model_monitor.describe_schedule()['MonitoringScheduleStatus'] != "Scheduled":
    print(f"Waiting until model monitoring status becomes Scheduled")
    time.sleep(3)
    
model_monitor.describe_schedule()

Waiting until model monitoring status becomes Scheduled
Waiting until model monitoring status becomes Scheduled
Waiting until model monitoring status becomes Scheduled
Waiting until model monitoring status becomes Scheduled


{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:906545278380:monitoring-schedule/from-idea-to-prod-model-monitor-schedule-2025-02-16-15-02-27',
 'MonitoringScheduleName': 'from-idea-to-prod-model-monitor-schedule-2025-02-16-15-02-27',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2025, 2, 16, 15, 2, 28, 889000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 2, 16, 15, 2, 39, 978000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2025-02-16-15-02-28-074',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'mlops-02-14-11-03-33-prod',
 'ResponseMetadata': {'RequestId': 'cdf7ee1d-8deb-4cb1-ae7a-2c1f176fef64',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cdf7ee1d-8deb-4cb1-ae7a-2c1f176fef64',
   'content-type': 'application/x-amz-json-1.1',
   'content-l

The endpoint has two scheduled monitors now, a data quality and a model quality monitor:

In [141]:
predictor.list_monitors()

[02/16/25 15:03:44] INFO     Ignoring unnecessary instance type: None.                            ]8;id=480079;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=25585;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#528\528]8;;\

[02/16/25 15:03:45] INFO     Ignoring unnecessary instance type: None.                            ]8;id=478897;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=847645;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#528\528]8;;\

### See model monitoring schedule executions

#### Inspect the lastest model monitor execution

In [142]:
# call describe_schedule to see the status of the latest completed execution
model_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:906545278380:monitoring-schedule/from-idea-to-prod-model-monitor-schedule-2025-02-16-15-02-27',
 'MonitoringScheduleName': 'from-idea-to-prod-model-monitor-schedule-2025-02-16-15-02-27',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2025, 2, 16, 15, 2, 28, 889000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 2, 16, 15, 2, 39, 978000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2025-02-16-15-02-28-074',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'mlops-02-14-11-03-33-prod',
 'ResponseMetadata': {'RequestId': 'd98979a8-41e9-4cb3-b393-8463e5187ccf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd98979a8-41e9-4cb3-b393-8463e5187ccf',
   'content-type': 'application/x-amz-json-1.1',
   'content-l

In [147]:
# List all _completed_ model monitor executions
model_mon_executions = model_monitor.list_executions()

In [148]:
model_mon_executions

In [149]:
assert len(model_mon_executions) > 0, 'Wait until you cross the hour boundary to see any model monitor executions!'

In [150]:
# See the details of the latest model monitor execution
latest_model_mon_execution = get_latest_monitor_execution(model_monitor)
execution_details = latest_model_mon_execution.describe()
execution_details

!Latest execution status: Completed
Latest execution result: CompletedWithViolations: Job completed successfully with 8 violations.
Report Uri: s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/prediction_baseline/reports/mlops-02-14-11-03-33-prod/from-idea-to-prod-model-monitor-schedule-2025-02-16-15-02-27/2025/02/16/16


{'ProcessingInputs': [{'InputName': 'constraints',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/prediction_baseline/results/constraints.json',
    'LocalPath': '/opt/ml/processing/baseline/constraints',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated'}},
  {'InputName': 'endpoint_input_1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/prediction_baseline/reports/merge/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/15',
    'LocalPath': '/opt/ml/processing/input_data/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/15',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'result',
    'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-906545278380/from-idea-to

#### See the execution reports

In [151]:
# Get the job output S3 uri
mon_job_output_s3_uri = execution_details["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
mon_job_output_s3_uri

's3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/prediction_baseline/reports/mlops-02-14-11-03-33-prod/from-idea-to-prod-model-monitor-schedule-2025-02-16-15-02-27/2025/02/16/16'

In [152]:
# Same S3 uri is accessible via the MonitoringExecution class
latest_model_mon_execution.output.destination

's3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/prediction_baseline/reports/mlops-02-14-11-03-33-prod/from-idea-to-prod-model-monitor-schedule-2025-02-16-15-02-27/2025/02/16/16'

In [153]:
# See the generated files - new statistics, constraints, and violations
!aws s3 ls {mon_job_output_s3_uri} --recursive

2025-02-16 16:12:09       1499 from-idea-to-prod/xgboost/prediction_baseline/reports/mlops-02-14-11-03-33-prod/from-idea-to-prod-model-monitor-schedule-2025-02-16-15-02-27/2025/02/16/16/constraint_violations.json
2025-02-16 16:12:09       1234 from-idea-to-prod/xgboost/prediction_baseline/reports/mlops-02-14-11-03-33-prod/from-idea-to-prod-model-monitor-schedule-2025-02-16-15-02-27/2025/02/16/16/constraints.json
2025-02-16 16:12:09       5815 from-idea-to-prod/xgboost/prediction_baseline/reports/mlops-02-14-11-03-33-prod/from-idea-to-prod-model-monitor-schedule-2025-02-16-15-02-27/2025/02/16/16/statistics.json


In [154]:
# Get the violation report from the MonitoringExecution class
last_execution_violations = latest_model_mon_execution.constraint_violations()

In [155]:
pd.json_normalize(last_execution_violations.body_dict["violations"]).head()

,constraint_check_type,description,metric_name
0,LessThanThreshold,Metric auc with 0.5836805555555556 was LessThanThreshold '0.7726853760190512',auc
1,LessThanThreshold,Metric precision with 0.0 was LessThanThreshold '0.6445783132530121',precision
2,LessThanThreshold,Metric truePositiveRate with 0.0 was LessThanThreshold '0.22291666666666668',truePositiveRate
3,LessThanThreshold,Metric f1 with 0.0 was LessThanThreshold '0.3312693498452013',f1
4,LessThanThreshold,Metric accuracy with 0.5844155844155844 was LessThanThreshold '0.8951201747997086',accuracy


You can access the violation report directly from the model monitor class:

In [156]:
# Use the ModelMonitor class
violations = model_monitor.latest_monitoring_constraint_violations()
pd.json_normalize(violations.body_dict["violations"]).head()

,constraint_check_type,description,metric_name
0,LessThanThreshold,Metric auc with 0.5836805555555556 was LessThanThreshold '0.7726853760190512',auc
1,LessThanThreshold,Metric precision with 0.0 was LessThanThreshold '0.6445783132530121',precision
2,LessThanThreshold,Metric truePositiveRate with 0.0 was LessThanThreshold '0.22291666666666668',truePositiveRate
3,LessThanThreshold,Metric f1 with 0.0 was LessThanThreshold '0.3312693498452013',f1
4,LessThanThreshold,Metric accuracy with 0.5844155844155844 was LessThanThreshold '0.8951201747997086',accuracy


#### See the merged datasets

In [157]:
# Get the S3 url to the merge datasets from the monitor job inputs
mon_job_merge_input_s3_uri = execution_details["ProcessingInputs"][1]["S3Input"]["S3Uri"]

mon_job_merge_bucket = mon_job_merge_input_s3_uri.split('/')[2]
mon_job_merge_prefix = '/'.join(mon_job_merge_input_s3_uri.split('/')[3:])

In [158]:
merge_files = get_file_list(mon_job_merge_bucket, mon_job_merge_prefix)

if merge_files:
    S3Downloader.download(f"s3://{mon_job_merge_bucket}/{merge_files[0]}", f"./tmp")

    print(f"Content of the merge file:")
    # Read the jsonl file and show two first objects
    with jsonlines.open(f"./tmp/{merge_files[0].split('/')[-1]}") as reader:      
        print(json.dumps(reader.read(), indent=2))
        print(json.dumps(reader.read(), indent=2))

Found 1 files in s3://sagemaker-us-east-1-906545278380/from-idea-to-prod/xgboost/prediction_baseline/reports/merge/mlops-02-14-11-03-33-prod/AllTraffic/2025/02/16/15
Content of the merge file:
{
  "eventVersion": "0",
  "groundTruthData": {
    "data": "0",
    "encoding": "CSV"
  },
  "captureData": {
    "endpointInput": {
      "data": "31,2,999,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0",
      "encoding": "CSV",
      "mode": "INPUT",
      "observedContentType": "text/csv"
    },
    "endpointOutput": {
      "data": "0.1255551427602768\n",
      "encoding": "CSV",
      "mode": "OUTPUT",
      "observedContentType": "text/csv; charset=utf-8"
    }
  },
  "eventMetadata": {
    "eventId": "88fdfecc-1a1a-4de9-b58e-70ce95a69c39",
    "inferenceTime": "2025-02-16T15:00:44Z"
  }
}
{
  "eventVersion": "0",
  "groundTruthData": {
    "data": "0",
    "encoding": "CSV"
  },
  "captureData": {
    "endpointInput": {
   

## Additional monitoring
Additionally to data and model quality monitoring with Model Monitor, you can use Amazon SageMaker Clarify to:
- [Monitor bias drift]
- [Monitor feature attribution drift]